In [17]:
#Card List

Cards={
    'Queen':{
        'Count': 4,
        'Value': 10,
        'Rank':12
        
    },
    'King':{
        'Count':4,
        'Value':10,
        'Rank':13
    },
    'Jack':{
        'Count':4,
        'Value':10,
        'Rank':11
    },
    '10':{
        'Count':4,
        'Value':10,
        'Rank':10
    },
        '9':{
        'Count':4,
        'Value':9,
        'Rank':9
    },
        '8':{
        'Count':4,
        'Value':8,
        'Rank':8
    },
        '7':{
        'Count':4,
        'Value':7,
        'Rank':7
    },
        '6':{
        'Count':4,
        'Value':6,
        'Rank':6
    },
        '5':{
        'Count':4,
        'Value':5,
        'Rank':5
    },
        '4':{
        'Count':4,
        'Value':4,
        'Rank':4
    },
        '3':{
        'Count':4,
        'Value':3,
        'Rank':3
    },
        '2':{
        'Count':4,
        'Value':2,
        'Rank':2
    },
}

In [18]:

# card class for hand
class Card():
    def __init__(self, cardname, cardvalue,cardrank,suit):
        
        self.cardname=cardname
        self.cardvalue=cardvalue
        self.cardrank=cardrank
        self.suit=suit
    def showcard(self):
        print ("""
        __________
        |{} 
        |         
        |{}
        |
        |__________
        """.format(self.cardname,self.suit), end =" ")

In [19]:

import random
#creates deck, and shuffles deck, keep track of dealed cards to shuffle them back into the deck when out of cards
class Deck():
    carddeck=[]
    droppedcards=[]
    def __init__  (self,decklist,suits):
        self.decklist=decklist
        self.suits=suits
    
    def makedeck(self):
        self.carddeck=[]
        for card in self.decklist:
            for count in range(self.decklist[card]['Count']):
                random.shuffle(self.suits)
                self.carddeck.append(Card(card,self.decklist[card]['Value'],self.decklist[card]['Rank'],self.suits[count%len(self.suits)]))
    
    def shuffle(self):
        randomcuts=random.randrange(50,200)
        for cuts in range(randomcuts):
            if len(self.carddeck)<15:
                shuffleamount=10
            else:
                shuffleamount=len(self.carddeck)-5
                
            randomcut=random.randrange(shuffleamount)
            randomcut2=random.randrange(randomcut,len(self.carddeck)-1)
            temp=self.carddeck[0:randomcut]
            temp2=self.carddeck[randomcut:randomcut2]
            self.carddeck=temp2+self.carddeck[randomcut2:]+temp
            random.shuffle(self.carddeck)
    def deal (self):
        self.droppedcards.append(self.carddeck.pop())
        return self.droppedcards[-1]
    
    def Newdeck(self):
        self.carddeck=self.carddeck+self.droppedcards
        self.droppedcards=[]
        self.shuffle()
            
        
        
            
        
        

In [20]:
#shuffle the deck multiple times
class MakeDeck(Deck):
    def startingdeck(self):
        self.makedeck()
        self.shuffle()
        self.shuffle()

In [68]:
#universal hand class
class Hand():
    cardlist=[]
    def add(self,card):
        self.cardlist.append(card)
    def remove (self,position):
        self.cardlist.pop(position)
        
    def showhand(self):
        for card in reversed(self.cardlist):
            card.showcard()
    def clear(self):
        self.cardlist=[]

In [69]:
#Hand specifically for blackjack
class Hand21(Hand):
    cardvalue=0
    cardlist=[]
    value=21
    def checkvalue(self):
        return self.cardvalue<=self.value
    def add(self,Deck):
        if self.checkvalue():
            
            self.cardlist.append(Deck.deal())
            self.cardvalue=self.cardvalue+self.cardlist[-1].cardvalue
            self.cardlist[-1].showcard()
            if self.cardvalue>self.value:
                print ('You Busted at a value of {}'.format(self.cardvalue))
        else:
            print ("Over Value {} Can't Take Another Card".format(self.value))
    def clear(self):
        self.cardlist=[]
        self.cardvalue=0
            
            
            
        


In [502]:
import collections
#Hand for texas holdem
#Stores current highest possible combination to display for User and give data to AI
class TexasHand(Hand):
    value=0
    highcard=0
    bestcards=[]
    possiblecards=[]
    cardlist=[]
    isstraight=0
    isflush=0
    cardsum=0
    def bestcardsum(self):
        self.cardsum=0
        for card in self.bestcards:
            self.cardsum=self.cardsum+card.cardrank
    def bestcardsclear(self):
        self.bestcards=[]
    
    def appendnamematch(self,name):

        for card in self.possiblecards:
            if card.cardname==name:
                self.bestcards.append(card)
    
    def findstraight(self):

        self.straightvalues=[]
        self.straightcards=[]
        for value in self.cardvalues:

            if len(self.straightvalues)==0:
                self.straightvalues.append(value)
            elif value==self.straightvalues[-1]+1 :
                self.straightvalues.append(value)
            elif len(self.straightvalues)>=5:
                True
            else:
                self.straightvalues=[]

        if len(self.straightvalues)>=5:
            self.isstraight=1
            if self.value<=3:
                self.value=4
                
        else:
            self.isstraight=0
    
    
    def checkifflush(self):
        self.isflush=0
        self.straightflushcheck=[]
        self.straightflushcards=[]
        if self.isstraight==1:

            for card in self.possiblecards:
                if card.cardrank in self.straightvalues:
                    self.straightflushcheck.append(card.suit)
                    self.straightflushcards.append(card)
                counter=collections.Counter(self.straightflushcheck)
                if (counter.most_common(1))[0][1]>4:
                    self.isflush=1
                    self.value=8
                    self.bestcards=self.straightflushcards
                else:
                    self.isflush=0
        else:

            self.cardsuits=[]
            for card in self.possiblecards:
                self.cardsuits.append(card.suit)
            counter=collections.Counter(self.cardsuits)

            if (counter.most_common(1))[0][1]>4:
                self.isflush=1
                if self.value<5:
                    self.bestcards=[]
                    self.value=5
                    self.flushvalue=(counter.most_common(1))[0][1]
                    for card in self.possiblecards:
                        self.bestcards.append(card)
            
            
                    
                
            
            
            
        
    
    def addhand(self,card):
        self.cardlist.append(card)
        self.addtable(card)        


    
    def addtable(self,card):
        self.possiblecards.append(card)
        
    
    def check(self):
        self.pairamount()
        self.straight()
        self.checkifflush()
        self.bestcardsum()


        
    def pairamount(self):
        self.cardvalues=[]
        for card in self.possiblecards:
            self.cardvalues.append(card.cardname)
        self.highcard=max(self.cardvalues)
        counter=collections.Counter(self.cardvalues)
        self.pairs= (counter.most_common(2))
        self.bestcardsclear()
        #4 of a kind
        if self.pairs[0][1]>=4:
            print ('4 Of Kind ')
            self.value=7
            self.appendnamematch(self.pairs[0][0])
        #full house
        elif len(self.pairs)>1:
            if self.pairs[0][1]==3 and self.pairs[1][1]>1:
                print (' Full House ')
                self.value=6
                self.appendnamematch(self.pairs[0][0])
                self.appendnamematch(self.pairs[1][0])
            #3 of a kind
            elif self.pairs[0][1]==3:
                print ('3 Of Kind ')
                self.value=3
                self.appendnamematch(self.pairs[0][0])
            #2 pair
            elif self.pairs[0][1]==2 and self.pairs[1][1]==2:
                print ('2 Pair')
                self.value=2
                self.appendnamematch(self.pairs[0][0])
                self.appendnamematch(self.pairs[1][0])
            #pair
            elif self.pairs[0][1]==2:
                print ('Pair ')
                self.value=1
        else:
       
            #3 of a kind
            if self.pairs[0][1]==3:
                print ('3 Of Kind ')
                self.value=3
                self.appendnamematch(self.pairs[0][0])
            #2 pair
            #pair
            elif self.pairs[0][1]==2:
                print ('Pair ')
                self.value=1            

    def straight(self):
        self.cardvalues=[]
        for card in self.possiblecards:
            self.cardvalues.append(card.cardrank)
        self.cardvalues=list(dict.fromkeys(self.cardvalues))
        self.cardvalues.sort()
        self.findstraight()
        
    def clear(self):
        self.possiblecards=[]
        self.cardlist=[]
        self.cardsum=0
        
        
    def showhand(self):
        print('Possible Cards')
        for card in reversed(self.possiblecards):
            card.showcard()    
            
        
        
        
        
        
        
        

In [503]:
#Supposed to be used for each individual game for online storage of players coming and going
class TexasGame():
    
    
    
    playerlist=[]
    
    def __init__(self,deck):
        self.deck=deck
        
    
    def addplayer( player):
        if len(playerlist)>3:
            print ("Too Many People Already Max is 4")
        else:
            self.playerlist.append(player)
    def removeplayer(i):
        self.playerlist[i]=TexasCPU()
        
        

In [507]:
#Player Class for Texas Holdem, gives players special functions only for the game

class PlayerTexas(Player21):
    hand=[]
    username=''
    lastbet=0
    current=0
    lastbet=0
    def __init__ (self,username):
        self.wallet=Wallet()
        self.username=username
    def hit (self,deck):
        if len(self.hand.cardlist)<=1:
            self.hand.add(deck.deal())
        else:
            print("Too Many Cards")
    
    def call(self,pot):
        betamount=this.current-this.lastbet
        if betamount>self.wallet.amount:
            betamount=self.wallet.amount
        self.wallet.remove(betamount)
        pot.add(betamount)
        
    
    def turn(self, turn, current,pot):
        if turn==1:
            self.current=current
            print ("Current Bet is {}".format(self.current))
            self.hand.showhand()
            betcallfold=int(input("It Is your Turn Bet:1 Call:2 Fold:3"))
            return betcallfold
    def bet(self,pot):
        
        betamount=int(input("How much more would you like to bet out of {}".format(self.wallet.amount)))
        if betamount>self.wallet.amount:
            print ("Invalid amount betting {}".format(self.wallet.amount))
            betamount=self.wallet.amount
        
        self.lastbest=this.current+betamount
        pot.add(betamount)
        self.wallet.remove(betamount)   
        return betamount
            
    def fold(self):
        return "FOLD"
            

In [504]:
#CPU for Texas Hold, logic for betting calling or folding
from random import randrange
class TexasCPU(PlayerTexas):
    whattodo=[]
    def __init__ (self):
        self.wallet=Wallet().add(100)
        self.username='CPU'
    def turn(self, turn, current,pot):
        self.hand.check()
        amounttospin=self.hand.value*10+10+(100-self.hand.value*10+10)
        ran=randrange(amounttospin)
        if ran<=self.hand.value*10+10:
            this.bet(pot)
        elif ran>self.hand.value*10+10:
            if ran=self.hand.value*10+10+(100-self.hand.value*10+10)*(2/3):
                this.fold()
            else:
                this.call(pot)
    def bet(self,pot,amount):
        betaamount=(self.hand.value*10+10)/100*wallet.amount
        if betaamount>wallet.amount:
            betaamount=wallet.amount
        self.wallet.remove(betamount)
        pot.add(betamount)
        
            
    #check score value*10+20 for chance of hit
    # hit amount random based on value 20+10*value percentage of max amount
    # fold 1/4 of call chance

In [505]:
#Class for each round of texas holdem to track turns and pot, and decide winner
class TexasRound():
    pot=Pot()
    playerstoskip=[]
    foldedplayers=[]
    turn=0
    player=0
    current=0
    turn=0
    tablehand=[]
    hands=[]
    def __init__ (self,playerlist,start,deck):
        self.playerlist=playerlist
        self.start=start
        self.player=start
        self.deck=deck
        self.tablehand=TexasHand()
    def play(self):
        while len(self.playerstoskip)<3 and self.turn<5:
            turnresult=self.playerlist[self.player%len(self.playerlist)].turn(1,this.current,pot)
            if turnresult=='FOLD':
                self.foldedplayers.append(self.playerlist[self.player%len(self.playerlist)])
                self.playerstoskip.append(self.playerlist[self.player%len(self.playerlist)])
            else:
                this.current=turnresult
            if self.player%len(self.playerlist)==self.start:
                self.turn=self.turn+1
            self.player=self.player+1

            self.tablehand.add(deck.deal())
        playerstotest=[x for x in self.playerlist if x not in self.foldplayers]
        for playerq in playerstotest:
            self.hands.append(palyerq.hand)
        
        
        
       
    def compareTexasHand(self):
        l=len(self.hands)
        i=0
        self.bestplayers=[]
        self.bestscore=0
        self.cardsum=0
        while i<l:
            if self.hands[i].value>bestscore:
                self.bestscore=hands[i].value
                self.bestplayers=[]
                self.bestplayers.append(i)
                cardsum=hands[i].cardsum
            elif self.hands[i].value==bestscore:
                if self.hands[i].cardsum>self.cardsum:
                    self.bestplayers=[]
                    self.bestplayers.append(i)
                    self.cardsum=hands[i].cardsum
                elif self.hands[i].cardsum==self.cardsum:
                    self.bestplayers.append(i)

            i=i+1
    def payout(self):
        payout=self.pot.remove()
        payouteach=payout/len(self.bestplayers)
        for player in self.bestplayers:
            players.wallet.add(payouteach)
        
        
    

In [27]:
#Player for blackjack
class Player21():
    amount=0
    def __init__ (self):
        self.wallet=Wallet()
    
    def givehand(self,hand):
        self.hand=hand
    def bet(self,amount,pot):
        pot.add(self.wallet.remove(amount))
    def hit(self,deck):
        self.hand.add(deck)
        if self.hand.cardvalue>21:
            self.check()
        else:
            self.amount=self.hand.cardvalue
        
    def stand(self):
        return self.hand.cardvalue
    
    def check(self):
        self.amount=self.hand.cardvalue
        print (self.amount)
        if self.amount>21:
            self.amount=-1
            print (self.amount)
        
        

In [28]:
#Dealer AI for blackjack, changes when they have to hit to  make it harder
from random import randrange
class Dealer21():
    def __init__  (self,deck):
        self.hand=Hand21()
        self.randomtop=randrange(13,20)
        self.deck=deck
    def dealed(self):
        self.hand=Hand21()
        while self.hand.cardvalue<self.randomtop:
            self.hand.add(self.deck)
            print (self.hand.cardvalue)
        if self.hand.cardvalue>21:
            return 0
        return self.hand.cardvalue

In [29]:
#Money for player, used to track overall amount of money for online
class Wallet():
    amount=0
    def add(self,amounts):
        self.amount=self.amount+amounts
    def remove(self,amounts):
        if amounts>self.amount:
            print ('You only have {} betting that amount'.format(self.amount))
            temp=self.amount
            self.amount=0
            return temp
        else:
            self.amount=self.amount-amounts
            return amounts
        
    

In [30]:
#Money Pot for the game
class Pot(Wallet):
    def remove(self):
        temp=self.amount
        self.amount=0
        return temp

In [ ]:
#Game of Blackjack
testdeck=MakeDeck(Cards,['Diamond','Hearts','Spades','Jack'])
testdeck.startingdeck()
Player1=Player21()
Player1.wallet.add(100)

Player1.givehand(Hand21())
playing=1
while playing==1:
    if len(testdeck.carddeck)<15:
        testdeck.Newdeck()
    Player1.hit(testdeck)
    Player1.hit(testdeck)
    bet_amount=int(input('How much would you like to bet out of {} '.format(Player1.wallet.amount)))
    pot=Pot()
    pot.add(Player1.wallet.remove(bet_amount))
    hitorstand=1
    while hitorstand==1:
        hitorstand=int(input("Current Value: {} Select HIT: 1 Stand: 2 ".format(Player1.hand.cardvalue)))
        if hitorstand==1:
            Player1.hit(testdeck)
            if Player1.amount==-1:
                Player1.hand.clear()
                playing=int(input("Bust! current wallet {}  To Play Again Enter 1 : ".format(Player1.wallet.amount)))
                hitorstand=0
                
        else:
            Player1.check()
            playervalue=Player1.stand()
            dealer=Dealer21(testdeck)
            dealervalue=dealer.dealed()
            dealer.hand.clear()
            print ("Player Got {}  Dealer Got {} ".format(playervalue,dealervalue))

            if dealervalue<playervalue:
                print ("YOU WON ")
                value=pot.remove()*2
                
                Player1.wallet.add(value)
    Player1.hand.clear()
    playing=int(input('Current Wallet {}  Play Again? 1 for Yes: '.format(Player1.wallet.amount)))
    print ("\n\n\n")
    if Player1.wallet.amount<1:
        print ("You Lost Goodbye")
        playing=0



        __________
        |10 
        |         
        |Diamond
        |
        |__________
         
        __________
        |2 
        |         
        |Spades
        |
        |__________
         How much would you like to bet out of 100 10
Current Value: 12 Select HIT: 1 Stand: 2 1

        __________
        |6 
        |         
        |Diamond
        |
        |__________
         Current Value: 18 Select HIT: 1 Stand: 2 2
18

        __________
        |10 
        |         
        |Jack
        |
        |__________
         10

        __________
        |4 
        |         
        |Hearts
        |
        |__________
         14

        __________
        |Queen 
        |         
        |Spades
        |
        |__________
         You Busted at a value of 24
24
Player Got 18  Dealer Got 0 
YOU WON 
Current Wallet 110  Play Again? 1 for Yes: 1





        __________
        |King 
        |         
        |Jack
        |
        |__________
  

In [493]:
testtexhand2=TexasHand()
testtexhand2.clear()
testtexhand2.add(Card('9',9,9,'Red'))
testtexhand2.add(Card('9',9,9,'Red'))
testtexhand2.add(Card('9',9,9,'Red'))
testtexhand2.add(Card('9',9,9,'Red'))
testtexhand2.add(Card('9',9,9,'Red'))

Pair 
3 Of Kind 
NAME
9
CARDNAME
9
OK
CARDNAME
9
OK
CARDNAME
9
OK
4 Of Kind 
NAME
9
CARDNAME
9
OK
CARDNAME
9
OK
CARDNAME
9
OK
CARDNAME
9
OK
4 Of Kind 
NAME
9
CARDNAME
9
OK
CARDNAME
9
OK
CARDNAME
9
OK
CARDNAME
9
OK
CARDNAME
9
OK


In [124]:
testdeck=MakeDeck(Cards,['Diamond','Hearts','Spades','Jack'])
testdeck.startingdeck()
Player1=PlayerTexas()
Player1.wallet.add(100)
Player1.givehand(TexasHand())

Player1.hit(testdeck)
Player1.hit(testdeck)

Too Many Cards
Too Many Cards
